In [1]:
import numpy as np

In [2]:
# Generater Spike time train using Poisson
# Refering "Poisson Model of Spike Generation" by Heeger (2000); https://www.cns.nyu.edu/~david/handouts/poisson.pdf.
def Spike_Time_Generater(rdt, dt, r_mean, kappa=1):  
    # rdt: original input (continuous)
    # dt: time bin length of the imput
    # r_mean: target mean firing rate (Hz)
    # kappa: kappa tune the spike interval (without changing the probability of spike count);
    #        First increase the target firing rate by kappa, then only expert every kappa-th spike.
    rdt = rdt / ( sum(rdt)/(len(rdt)*dt) ) *r_mean *kappa
                #original mean firing rate
    Spike_time = []
    
    timeBinCounter = 0 # time bin index
    p = 1 # probability of no spike in a bin.

    while True:

        # Use a random number to determine if there is a spike in the bin; if not, accumalate the probability of not firing (lowering "p"). 
        # The while loop stops when a spike appears.
        theRandomNumber = 1-np.random.rand()
        while (theRandomNumber < p and timeBinCounter < len(rdt)): 
            p *= np.exp(-rdt[timeBinCounter]) 
            timeBinCounter += 1
        if timeBinCounter >= len(rdt): # End the funciton when current time exceeds the total time.
            return Spike_time[::kappa]

        # Resume the exceeding probability (from "theRandomNumber" to "p") into the time unit, 
        # subtract it from the current time to determine the exact firing time and renew "p" with it. 
        remainer_c = -np.log(p/theRandomNumber)/rdt[timeBinCounter-1]
        Spike_time.append(dt*(timeBinCounter-remainer_c))
        p = np.exp(-remainer_c*rdt[timeBinCounter-1]) 

In [3]:
# activation function
def NL(inp, theta = 0):
    y = inp.copy()
    y[y<theta] = theta
    return y-theta

In [4]:
# activation function
def nA(x,theta=0):
    y = np.zeros(len(x))
    for i in range(len(x)):
        if x[i]- theta>0:
            y[i]= 1
    return(y)

# activation function
def nA2(x,theta1, theta2):
    y = np.zeros(len(x))
    for i in range(len(x)):
        if x[i]- theta2>0:
            y[i]= 2
        elif x[i]- theta1>0:
            y[i]= 1
    return(y)

In [5]:
def SG(w, dt, theta_star=0, r_mean=5, kappa = 10, thetaType = 'rel'):
    T = np.arange(len(w)+1)*dt
    if thetaType == 'rel':
        wstate, _ = np.histogram( Spike_Time_Generater( NL(w,np.mean(w)+theta_star*np.std(w)) , dt, r_mean, kappa), T)
    elif thetaType == 'abs':
        wstate, _ = np.histogram( Spike_Time_Generater( NL(w,theta_star) , dt, r_mean, kappa), T)
    return wstate